In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import string
import re
#import gzip
import collections
from nltk.corpus import stopwords
from gensim import corpora,models
import json
from collections import defaultdict
%matplotlib inline

In [4]:
AMAZON_DATA_FILE_PATH = "/Users/siva/Documents/CS506/datasets/Books_5.json"
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield eval(l)

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
texts = []
count = 0
for i,l in enumerate(parse(AMAZON_DATA_FILE_PATH)):
    tokens = tokenizer.tokenize(l["reviewText"].lower())
    tokens = [e for e in tokens if e not in stop_words]
    #stemmed_tokens = [stemmer.stem(s) for s in tokens]
    if len(tokens)>200:
        count += 1
        texts.append(tokens)
    if count == 1:
        break
print(len(texts))

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(len(corpus))
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

print(ldamodel.print_topics(num_topics=2, num_words=3))

In [2]:
YELP_REVIEWS_DATA = "/Users/siva/Documents/CS506/datasets/yelp_dataset/review.json"
business_reviews = defaultdict(list)
with open(YELP_REVIEWS_DATA,"r") as f_json:
    for i,line in enumerate(f_json):
        review = json.loads(line)
        business_reviews[review["business_id"]].append(review["text"])
        if i % 100000 == 0:
            print(i)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000


In [5]:
print(sum([1 for each in business_reviews if len(business_reviews[each])>100]))

10274


In [92]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
restaurants_with_topics = {}
NO_OF_WORDS_IN_EACH_REVIEW_THRESHOLD = 3
NO_OF_SENTENCES_IN_EACH_REVIEW_THRESHOLD = 7
NO_OF_TOPICS = 2

for i,each_business in enumerate(business_reviews):
    if i % 10 == 0:
        print(i)
    if len(business_reviews[each_business]) > 100:
        reviews_with_topics = defaultdict(list)
        for each_review in business_reviews[each_business]:
            sentences_each_review = each_review.split(".")
            texts = []
            for each_sentence in sentences_each_review:
                tokens = tokenizer.tokenize(each_sentence.replace("\n","").lower())
                tokens = [e for e in tokens if e not in stop_words]
                if len(tokens)>NO_OF_WORDS_IN_EACH_REVIEW_THRESHOLD:
                    texts.append(tokens)
            if len(texts) > NO_OF_SENTENCES_IN_EACH_REVIEW_THRESHOLD:
                dictionary = corpora.Dictionary(texts)
                corpus = [dictionary.doc2bow(text) for text in texts]
                ldamodel = models.ldamodel.LdaModel(corpus, num_topics=NO_OF_TOPICS, id2word = dictionary, passes=20)
                topic = ldamodel.show_topics(num_topics = 1,num_words = 1,formatted = False)[0][1][0][0]
                reviews_with_topics[topic].append(each_review)
        restaurants_with_topics[each_business] = reviews_with_topics

In [96]:
with open("results.json","w") as write_json:
    for each in restaurants_with_topics:
        json.dump(restaurants_with_topics[each],write_json)